In [55]:
# Profile Extractor

This class provide functions that allow the retrieval of informations about a musician. Below are some useful methods for retrieving the Genre, Discography or Provenance. For this, we will use the SpotifyAPI and Wikipedia.

In [56]:
import numpy as np
import requests
import json
import itertools
import bs4

associates = []

listGenres = "https://en.wikipedia.org/wiki/List_of_electronic_music_genres"
contentWIKI = requests.get(listGenres).text

In [57]:
def updateDictionnary(genres,dictionnary,debug = False):
    for genre in genres:
        genre = genre.lower()
        if(not genre in dictionnary):
            #gs = itertools.combinations(genre.split(" "),i)
            spacesplit = genre.split(" ")
            gs = []
            for a in spacesplit:
                linesplit = a.split("-")
                for l in linesplit:
                    gs.append(str(l))
                
            mainGenre = None
            for g in gs:
                #g = str(g).replace("', '"," ").replace("')","").replace("('","").replace("',)","")
                if(g in dictionnary):
                    mainGenre = dictionnary[g]
                    dictionnary.update({genre : mainGenre})   
                    if(debug):            
                        print("added : dic{"+genre+" : "+mainGenre+"}")
                    #return dictionnary
                else:
                    if(len(g)>1):
                        dictionnary.update({g : g})
                        if(debug):
                            print("added : dic{"+g+" : "+g+"}")
                #if(genre in dictionnary):
                     #   return dictionnary
                    #else:
                       # dictionnary.update({genre : genre)
        if(genre not in dictionnary):  
            if(debug):
                print("missing genre : "+genre)
            dictionnary.update({genre : genre})

    return dictionnary

In [78]:
def createDictionnary(debug = False):
    #wikiURL = "https://en.wikipedia.org/wiki/"+str(genre)
    listGenres = "https://en.wikipedia.org/wiki/List_of_electronic_music_genres"
    
    #content = requests.get(listGenres).text
    soup = bs4.BeautifulSoup(contentWIKI,"html5lib")
    mainList = []
    subgenres = []
    dic = {}
    main = ""
    
    for div in soup.findAll(attrs={"style" : "-moz-column-width: 20em; -webkit-column-width: 20em; column-width: 20em;"}):   
        if(div.attrs['style'] == "-moz-column-width: 20em; -webkit-column-width: 20em; column-width: 20em;"):
             if('style' in div.attrs):
                uls = div.find('ul')
                if(uls is not None):
                    for groups in uls:
                        for a in groups:
                            b = str(a)
                            soup2 = bs4.BeautifulSoup(b,"html5lib")
                            if(len(soup2.text)>0 and soup2.text.count("\n")<2):
                                mainList.append(soup2.text)
                                main = soup2.text.replace("\n","")
                                dic.update({main.lower() : main.lower()})
                                if(debug):print("MAIN "+str(len(main))+" "+main)
                                
                            if("ul" in b and len(b)>0):
                                soup2 = bs4.BeautifulSoup(b,"html5lib")
                                if(len(soup2.text)>0):
                                    t = soup2.text
                                    lines = t.split("\n")
                                    for genre in lines:
                                        if(len(genre)>0):
                                            if(debug):
                                                print(genre +"-"+main)
                                            if(debug):
                                                print("...")
                                            dic.update({genre.lower() : main.lower()})
                                
       
    ##Manual Add 
    dic.update({"trap francais" : "rap"})
    dic.update({"catstep":"drum and bass"})
    dic.update({"dance pop" : "pop"})
    dic.update({"complextro" : "uk garage"})
    dic.update({"hip hop" : "hip hop"})
    dic.update({"hip" : "hip hop"})
    dic.update({"hop" : "hip hop"})
    dic.update({"rock" : "rock"})
    dic.update({"rap" : "rap"})
    dic.update({"filthstep" : "uk garage"})
    
    return dic
            

In [79]:
def findMainGenre(genre,dic):
    return dic[genre.lower()]

In [80]:
def sanitize(genres):
    if(genres is None):
        return genre
    else:
        gen = []
        
        nations = ["italian","autralian","french","english","greek","latin"]
        for genre in genres:
            gs = genre.split(" ")
            valid = True
            for g in gs:
                if(g in nations):
                     valid = False
            if(valid):
                gen.append(genre)
    return gen

In [81]:
def getMaxGenre(genres,debug = False):
    dicnum = {}
    for g in genres :
        dicnum.update({g : genres.count(g)})
   
    maxKey = 0
    maxEntry = None
    if(debug):
        print(dicnum)
    for entry in dicnum :
        if dicnum[entry]>maxKey and entry is not None:
            maxEntry = entry
            maxKey = dicnum[entry]
    
    return maxEntry     

In [62]:
dic = createDictionnary()
findMainGenre("chillstep",dic)

'uk garage'

In [63]:
def mainGenres(genres,dictionnary=None, debug = False):
    
        dictionnary = updateDictionnary(genres,dictionnary, debug)
        
        mainGenreAssociates = []
        for genre in genres :
            #Update the list with the new genre
            #if(genre not in dictionnary):
               # dictionnary = updateDictionnary(genre,dictionnary, debug)
                    
    
             #mainGenreAssociates.append(associates[genre])
            if(debug):
                print("for "+str(genre)+" found "+dictionnary[genre])
            mainGenreAssociates.append(dictionnary[genre])
            

        return mainGenreAssociates

In [64]:
def getGenresFromSpotify(Artist):
    SpotifySearchURL = "https://api.spotify.com/v1/search"
    
    #arguments
    args = {
         'q' : str(Artist),
        'type' : 'artist'
    }
    
    #Query 
    content = json.loads(requests.get(SpotifySearchURL, params = args).text)
    
    genres = content['artists']['items'][0]['genres']
    return genres

In [65]:
def getGenre(Artist,dictionnary = None, Year=None, debug= False):
    '''Get the genre of an artist at this period.If period is None, 
    gets the most representative genre of the artist
    across time.'''
    if(dictionnary is None):
        dictionnary = createDictionnary()
    
    genres = getGenresFromSpotify(Artist)
    genres = sanitize(genres)
    
    MG = mainGenres(genres,dictionnary, debug)
    
    #print("Genres : "+str(genres))
    #print("mainGenres : "+str(MG))
    
    
    mainGenre = getMaxGenre(MG,debug)
    
    return mainGenre
    
    

In [66]:
#genre = getGenre("Michael Jackson",dictionnary)#, debug = True)
dictionnary = createDictionnary()

artists = ["Michael Jackson", "Renaud", "IAM", "Elvis Presley", "Shakira", 
           "Panteros666", "Aqua", "Paolo Conte", "DJ Splash", "Phil Collins", "U2", "Pendulum", "Prodigy",
          "Daddy Yankee","Mika","Glitch-Mob", "Yannick", "Indochine", "Peter Gabriel", "Bob Marley", "Damian Marley",
          "Skrillex", "Mija", "Zaho", "Jul"]
for a in artists:
    print("{name} - {genre} ".format(name = a, genre = getGenre(a,dictionnary)))

#print(genre)

Michael Jackson - pop 
Renaud - chanson 
IAM - rap 
Elvis Presley - rock 
Shakira - pop 
Panteros666 - techno 
Aqua - pop 
Paolo Conte - chanson 
DJ Splash - hands up 
Phil Collins - rock 
U2 - rock 
Pendulum - drum and bass 
Prodigy - breakbeat 
Daddy Yankee - reggaeton 
Mika - pop 
Glitch-Mob - uk garage 
Yannick - orchestral 
Indochine - chanson 
Peter Gabriel - rock 
Bob Marley - reggae 
Damian Marley - reggae 
Skrillex - uk garage 
Mija - house music 
Zaho - rap 
Jul - rap 


In [67]:
def getMeanGenre(genres):
    '''If an artist has a lot of genres, try to find the more representative'''

In [68]:
def getDiscography(Artist,Year = None):
    return

In [82]:
getGenre("Pendulum",createDictionnary(),debug=True)

added : dic{australian : australian}
added : dic{dance : dance}
missing genre : australian dance
added : dic{edm : edm}
added : dic{electronic : electronic}
for australian dance found australian dance
for brostep found uk garage
for catstep found drum and bass
for drum and bass found drum and bass
for edm found edm
for electro house found house music
for electronic found electronic
{'drum and bass': 2, 'uk garage': 1, 'electronic': 1, 'australian dance': 1, 'house music': 1, 'edm': 1}


'drum and bass'

In [83]:
getGenre("Indochine",createDictionnary(),debug=True)

added : dic{chanson : chanson}
added : dic{francoton : francoton}
for chanson found chanson
for francoton found francoton
{'chanson': 1, 'francoton': 1}


'chanson'

In [84]:
getGenresFromSpotify("Indochine")

['chanson', 'francoton', 'french pop', 'french rock']

In [93]:
getGenre("Ben Klock",None,debug=True)

added : dic{german : german}
added : dic{german techno : techno}
added : dic{microhouse : microhouse}
added : dic{minimal : minimal}
added : dic{tech : tech}
added : dic{house : house}
missing genre : minimal tech house
for detroit techno found techno
for german techno found techno
for microhouse found microhouse
for minimal tech house found minimal tech house
for minimal techno found techno
for tech house found house music
for techno found techno
{'techno': 4, 'house music': 1, 'minimal tech house': 1, 'microhouse': 1}


'techno'